In [1]:
!pip install gensim

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.24
    Uninstalling Cython-0.29.24:
      Successfully uninstalled Cython-0.29.24


In [2]:
!pip install python-Levenshtein

  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp38-cp38-win_amd64.whl size=83809 sha256=2713043f6bdf036e111d3aa6ddc7307d7b1a1d12c6151abe235a555614d88a3f
  Stored in directory: c:\users\21355\appdata\local\pip\cache\wheels\d7\0c\76\042b46eb0df65c3ccd0338f791210c55ab79d209bcc269e2c7
Successfully built python-Levenshtein


In [3]:
import pandas as pd 

In [5]:
df = pd.read_json("Cell_Phones_and_Accessories_5.json",lines=True)

In [6]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [10]:
import gensim 

We are going to use gensim for the NLP model because it's less complicated and contains better fonctions 

In [18]:
# for the pre-processing of the text we will use the .simple_preprocess() fonction  
gensim.utils.simple_preprocess("They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again")



['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

The .simple_preprocessing() fonction does some text cleaning on the sentence in input , it tokenize the words and eliminate all the ponctuation in the sentence, also it convert all in lowercase

we will apply the fonction to all the column and we will get as result a list of words for every instance 

In [ ]:
reivew_text = df["reviewText"].apply(gensim.utils.simple_preprocess)

In [19]:
reivew_text

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [23]:
# like we saw previousely in the theory, for browsing all the text and creating our training data (Xs , and Ys) we need to set a window to tell the model how much words will pick 
model = gensim.models.Word2Vec(
    window=10, # we will take 10 words after and before our target word
    min_count= 2 , # this for telling the model to take the sentences that has num of words equal or greater than 2 
    workers = 4 , #cpu Threads to train the model 
)

In [24]:
# now we are going to build a vocab based on our review texts 
# the vocab contains a unique list of words 
# and for that we have some built-in fonction 
model.build_vocab(reivew_text,progress_per=1000)

In [35]:
# now we are going to train the model 
# by default the model contains pre-defined fonctions and also has a number of epochs for the training can be found in the model.epochs variable
model.train(reivew_text,total_examples=model.corpus_count,epochs=model.epochs)

(61505338, 83868975)

In [36]:
# for avoid the training process and all the cleaning data staff we are going to save the model in order to use it in other applicaitons
model.save("./Word2VecModel.model")

In [46]:
# now we are going to do some experiments 
# there is a fonction called .wv.most_similar(word) which give us all the words that are similar to "word",
# we have also the similarity score for every word that is similar to "word"

model.wv.most_similar("good")

[('decent', 0.8130378127098083),
 ('great', 0.7785266041755676),
 ('nice', 0.6918162107467651),
 ('fantastic', 0.6872422099113464),
 ('excellent', 0.6382060050964355),
 ('exceptional', 0.6362926959991455),
 ('superb', 0.6221593618392944),
 ('awesome', 0.5958556532859802),
 ('outstanding', 0.586286187171936),
 ('amazing', 0.5829739570617676)]

In [51]:
model.wv.similarity(w1="rain",w2="great")

-0.073471576